In [147]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [35]:
INTAKES_URL = "https://data.austintexas.gov/resource/wter-evkm.json"
PARAMS = {
    "animal_type": "Dog",
    "$$app_token": "aBq5AbxH1zKfProvzcMh0h4Dm"
}

In [44]:
r = requests.get(INTAKES_URL, params=PARAMS)
df = pd.DataFrame(r.json())
df = df.drop(columns=['datetime2']).fillna('None')
len(df)

1000

In [194]:
df.head(10)

,animal_id,datetime,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,name
0,A826026,2020-11-17T17:25:00.000,5347 Mission Circle in Austin (TX),Stray,Normal,Dog,Intact Female,1 month,Pit Bull,Black,None
1,A826027,2020-11-17T17:25:00.000,5347 Mission Circle in Austin (TX),Stray,Normal,Dog,Intact Female,1 month,Pit Bull,Brown Brindle,None
2,A826025,2020-11-17T17:25:00.000,5347 Mission Circle in Austin (TX),Stray,Normal,Dog,Intact Male,1 month,Pit Bull,Black,None
3,A826016,2020-11-17T16:21:00.000,5716 Malarkey Rd in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Chihuahua Shorthair/Boston Terrier,Gray/White,None
4,A826023,2020-11-17T17:25:00.000,5347 Mission Circle in Austin (TX),Stray,Normal,Dog,Intact Male,1 month,Pit Bull,Black,None
5,A826024,2020-11-17T17:25:00.000,5347 Mission Circle in Austin (TX),Stray,Normal,Dog,Intact Female,1 month,Pit Bull,Black,None
6,A826022,2020-11-17T17:25:00.000,5347 Mission Circle in Austin (TX),Stray,Normal,Dog,Intact Female,3 years,Pit Bull,Black,None
7,A826014,2020-11-17T16:01:00.000,4501 Depew Ave in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Australian Shepherd/Labrador Retriever,Black,None
8,A826011,2020-11-17T15:58:00.000,1071 Clayton Lane in Austin (TX),Stray,Sick,Dog,Unknown,6 months,Catahoula Mix,Brown Brindle/White,None
9,A826004,2020-11-17T15:12:00.000,12102 Clickett Cove in Austin (TX),Stray,Nursing,Dog,Intact Female,2 years,Labrador Retriever Mix,Tricolor,None


In [193]:
# =============
# DATA CLEANING
# =============
 
# Create empty lists for new columns
intake_datetime = []
found_address = []
found_city = []
found_state = []
sex = []
birthday = []
age = []
primary_breed = []
secondary_breed = []
mixed_breed = []
primary_color = []
secondary_color = []
name = []

for i in df.index:
    # Clean datetime column
    intake_dt = datetime.strptime(df.iloc[i]['datetime'].replace(".000", ""), '%Y-%m-%dT%H:%M:%S')
    intake_datetime.append(intake_dt)

    # Clean location column
    if len(df.iloc[i]['found_location'].split(" in ")) > 1:
        found_address.append(df.iloc[i]['found_location'].split(" in ")[0])
        found_city.append(df.iloc[i]['found_location'].split(" in ")[1].split(" (")[0])
        found_state.append(df.iloc[i]['found_location'].split(" in ")[1].split(" (")[1].replace(")", ""))
    elif "(TX)" in df.iloc[i]['found_location'].split(" in ")[0]:
        found_address.append('None')
        found_city.append(df.iloc[i]['found_location'].split(" in ")[0].split(" (")[0])
        found_state.append(df.iloc[i]['found_location'].split(" in ")[0].split(" (")[1].replace(")", ""))
    else:
        found_address.append(df.iloc[i]['found_location'])
        found_city.append('None')
        found_state.append('None')

    # Clean sex column
    sex.append(df.iloc[i]['sex_upon_intake'].replace("Intact ", "").replace("Neutered ", "").replace("Spayed ", ""))

    # Clean age upon intake column
    intake_age = df.iloc[i]['age_upon_intake'].split(" ")
    if intake_age[0].isnumeric():
        num = int(intake_age[0])
    else:
        num = "Unknown"
    if len(intake_age) > 1:
        duration = (
            intake_age[1]
            .replace("years", "year").replace("year", "years")
            .replace("months", "month").replace("month", "months")
            .replace("weeks", "week").replace("week", "weeks")
            .replace("days", "day").replace("day", "days")
            )
    else:
        duration = "Unknown"
    # Determine birthday
    if num == "Unknown" or duration == "Unknown":
        bday = "Unknown"
        birthday.append(bday)
    else:
        bday = intake_dt - relativedelta(**{duration: num})
        birthday.append(bday)
    # Determine age as of today's date
    if isinstance(intake_dt, datetime) and isinstance(bday, datetime):
        age_today = relativedelta(datetime.today(), bday)
        age.append(age_today)
    else:
        age.append("Unknown")
    
    # Clean breed column
    if len(df.iloc[i]['breed'].split("/")) > 1:
        primary_breed.append(df.iloc[i]['breed'].split("/")[0])
        secondary_breed.append(df.iloc[i]['breed'].split("/")[1])
        mixed_breed.append(True)
    else:
        primary_breed.append(df.iloc[i]['breed'])
        secondary_breed.append('None')
        mixed_breed.append(False)

    # Clean color column
    if len(df.iloc[i]['color'].split("/")) > 1:
        primary_color.append(df.iloc[i]['color'].split("/")[0])
        secondary_color.append(df.iloc[i]['color'].split("/")[1])
    else:
        primary_color.append(df.iloc[i]['color'])
        secondary_color.append('None')

    # Clean name column
    name.append(df.iloc[i]['name'].replace("*", ""))

# Compile clean dataframe
df2 = pd.DataFrame({
    "animal_id": df["animal_id"],
    "intake_datetime": intake_datetime,
    "found_address": found_address,
    "found_city": found_city,
    "found_state": found_state,
    "intake_type": df["intake_type"],
    "intake_condition": df["intake_condition"],
    "sex": sex,
    "birthday": birthday,
    "age": age,
    "mixed_breed": mixed_breed, 
    "primary_breed": primary_breed, 
    "secondary_breed": secondary_breed,
    "primary_color": primary_color,
    "secondary_color": secondary_color,
    "name": name, 
    })

df2.head(20)

,animal_id,intake_datetime,found_address,found_city,found_state,intake_type,intake_condition,sex,birthday,age,mixed_breed,primary_breed,secondary_breed,primary_color,secondary_color,name
0,A826026,2020-11-17 17:25:00,5347 Mission Circle,Austin,TX,Stray,Normal,Female,2020-10-17 17:25:00,"relativedelta(months=+1, hours=+22, minutes=+5...",False,Pit Bull,None,Black,None,None
1,A826027,2020-11-17 17:25:00,5347 Mission Circle,Austin,TX,Stray,Normal,Female,2020-10-17 17:25:00,"relativedelta(months=+1, hours=+22, minutes=+5...",False,Pit Bull,None,Brown Brindle,None,None
2,A826025,2020-11-17 17:25:00,5347 Mission Circle,Austin,TX,Stray,Normal,Male,2020-10-17 17:25:00,"relativedelta(months=+1, hours=+22, minutes=+5...",False,Pit Bull,None,Black,None,None
3,A826016,2020-11-17 16:21:00,5716 Malarkey Rd,Austin,TX,Stray,Normal,Female,2018-11-17 16:21:00,"relativedelta(years=+2, hours=+23, minutes=+55...",True,Chihuahua Shorthair,Boston Terrier,Gray,White,None
4,A826023,2020-11-17 17:25:00,5347 Mission Circle,Austin,TX,Stray,Normal,Male,2020-10-17 17:25:00,"relativedelta(months=+1, hours=+22, minutes=+5...",False,Pit Bull,None,Black,None,None
5,A826024,2020-11-17 17:25:00,5347 Mission Circle,Austin,TX,Stray,Normal,Female,2020-10-17 17:25:00,"relativedelta(months=+1, hours=+22, minutes=+5...",False,Pit Bull,None,Black,None,None
6,A826022,2020-11-17 17:25:00,5347 Mission Circle,Austin,TX,Stray,Normal,Female,2017-11-17 17:25:00,"relativedelta(years=+3, hours=+22, minutes=+51...",False,Pit Bull,None,Black,None,None
7,A826014,2020-11-17 16:01:00,4501 Depew Ave,Austin,TX,Stray,Normal,Male,2018-11-17 16:01:00,"relativedelta(years=+2, days=+1, minutes=+15, ...",True,Australian Shepherd,Labrador Retriever,Black,None,None
8,A826011,2020-11-17 15:58:00,1071 Clayton Lane,Austin,TX,Stray,Sick,Unknown,2020-05-17 15:58:00,"relativedelta(months=+6, days=+1, minutes=+18,...",False,Catahoula Mix,None,Brown Brindle,White,None
9,A826004,2020-11-17 15:12:00,12102 Clickett Cove,Austin,TX,Stray,Nursing,Female,2018-11-17 15:12:00,"relativedelta(years=+2, days=+1, hours=+1, min...",False,Labrador Retriever Mix,None,Tricolor,None,None


In [195]:
# Extract age as a string of years, months, weeks (sample for the first 20 rows in df2)
[f"{age.years} years, {age.months} months, {age.weeks} weeks" for age in df2.head(20)['age'] if isinstance(age, relativedelta)]

['0 years, 1 months, 0 weeks',
 '0 years, 1 months, 0 weeks',
 '0 years, 1 months, 0 weeks',
 '2 years, 0 months, 0 weeks',
 '0 years, 1 months, 0 weeks',
 '0 years, 1 months, 0 weeks',
 '3 years, 0 months, 0 weeks',
 '2 years, 0 months, 0 weeks',
 '0 years, 6 months, 0 weeks',
 '2 years, 0 months, 0 weeks',
 '0 years, 9 months, 0 weeks',
 '0 years, 9 months, 0 weeks',
 '0 years, 6 months, 0 weeks',
 '6 years, 0 months, 0 weeks',
 '7 years, 0 months, 0 weeks',
 '2 years, 0 months, 0 weeks',
 '2 years, 4 months, 2 weeks',
 '2 years, 0 months, 0 weeks',
 '2 years, 0 months, 0 weeks',
 '2 years, 0 months, 0 weeks']